In [151]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd

#### For only one movie

In [2]:
# Load web page
webPage = requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')

# Convert to beatiful soup object
soup = bs(webPage.content)

# Print html
contents = soup.prettify()


In [3]:
# Finding the box with interest information
infoBox = soup.find(class_='infobox vevent')
infoRows = infoBox.find_all('tr')
for row in infoRows:
    print(row.prettify())

<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  Toy Story 3
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Toy_Story_3_poster.jpg" title="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3.">
   <img alt="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3." class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/6/69/Toy_Story_3_poster.jpg" width="220"/>
  </a>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Directed by
 </th>
 <td class="

In [14]:
# Scrapping info into dictionary - ToyStory 3
movieInfo = {}

def get_content_value(rowData):
    if rowData.find('li'):
        return [li.get_text(' ',strip=True).replace('\xa0',' ') for li in rowData.find_all('li')]
    else:
        return rowData.get_text(' ',strip=True).replace('\xa0',' ')

for index, row in enumerate(infoRows):
    if index == 0:
        movieInfo['title'] = row.find('th').get_text(' ',strip=True)
    elif index == 1:
        continue
    else: 
        contentKey = row.find('th').get_text(' ',strip=True)
        contentValue = get_content_value(row.find('td'))
        movieInfo[contentKey] = contentValue

#### For every movie on list

In [146]:
def get_content_value(rowData):
    if rowData.find('li'):
        return [li.get_text(' ',strip=True).replace('\xa0',' ') for li in rowData.find_all('li')]
    else:
        return rowData.get_text(' ',strip=True).replace('\xa0',' ')

def get_info_box(url):
    #headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    #timeout = 5
    webPage = requests.get(url)
    soup = bs(webPage.content)
    infoBox = soup.find(class_='infobox vevent')
    infoRows = infoBox.find_all('tr')
    
    movieInfo = {}

    for index, row in enumerate(infoRows):
        if index == 0:
            movieInfo['title'] = row.find('th').get_text(' ',strip=True)
        elif index == 1:
            continue
        else: 
            contentKey = row.find('th').get_text(' ',strip=True)
            contentValue = get_content_value(row.find('td'))
            movieInfo[contentKey] = contentValue
    return movieInfo

In [144]:
# Dont execute again

# Scrapping every movie link
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
webPage = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(webPage.content)
movieList = soup.select('.wikitable.sortable i a') # i = italic, a = link element(in this case avoids exceptions of missing links)

movieDataset = []
errors = []

for index,item in enumerate(movieList):
    if index % 100 == 0:
        print('Actual index: ', index)
        print('Until finish: ', len(movieList)-index)
    try:
        url = 'https://en.wikipedia.org' + movieList[index]['href']  
        movieDataset.append(get_info_box(url))
    except Exception as e:
        print(index)
        print(e)
        errors.append(index)

Actual index:  0
Until finish:  454
43
'NoneType' object has no attribute 'find'
48
'NoneType' object has no attribute 'find'
63
('Connection aborted.', TimeoutError(10060, 'Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu', None, 10060, None))
Actual index:  100
Until finish:  354
112
'NoneType' object has no attribute 'get_text'
124
'NoneType' object has no attribute 'find_all'
145
'NoneType' object has no attribute 'find'
Actual index:  200
Until finish:  254
247
('Connection aborted.', TimeoutError(10060, 'Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu', None, 10060, None))
258
'NoneType' object has no attribute 'get_text'
288
'NoneType' object has no attribute 'get_text'
Actual index:  300
Until fini

In [154]:
# Dont execute again
# Saving to json file
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
# Loading
def load_data(title, data):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

#### Cleaning Data
- Clean up references "[1]"
- Convert running time into an integer
- Convert dates into datetime object
- Split up long strings
- Convert budget & box office to numbers

In [152]:
# Loading json file
dataset = pd.read_json('movieDataset.json')